In [9]:
from ipywidgets import IntProgress
import requests
from keybert import KeyBERT
from SPARQLWrapper import SPARQLWrapper, JSON
from rdflib import Graph 

In [10]:
doc = [
    """
       An interplay between Ca2+/calmodulin-dependent protein kinase IIδc (CaMKIIδc) and late
       Na+ current (INaL) is known to induce arrhythmias in the failing heart. Here, we elucidate 
       the role of the sodium channel isoform NaV1.8 for CaMKIIδc-dependent proarrhythmia. 
       In a CRISPR-Cas9-generated human iPSC-cardiomyocyte homozygous knock-out of NaV1.8, 
       we demonstrate that NaV1.8 contributes to INaL formation. In addition, we reveal a direct 
       interaction between NaV1.8 and CaMKIIδc in cardiomyocytes isolated from patients with heart 
       failure (HF). Using specific blockers of NaV1.8 and CaMKIIδc, we show that NaV1.8-driven INaL
       is CaMKIIδc-dependent and that NaV1.8-inhibtion reduces diastolic SR-Ca2+ leak in human failing
       cardiomyocytes. Moreover, increased mortality of CaMKIIδc-overexpressing HF mice is reduced when 
       a NaV1.8 knock-out is introduced. Cellular and in vivo experiments reveal reduced ventricular
       arrhythmias without changes in HF progression. Our work therefore identifies a proarrhythmic
       CaMKIIδc downstream target which may constitute a prognostic and antiarrhythmic strategy.
      """ , 
        """
         Patients with heart failure and preserved ejection fraction (HFpEF) have a high burden of symptoms and functional
         limitations, and have a poor quality of life. By targeting cardiometabolic abmormalities, sodium glucose cotransporter
         2 (SGLT2) inhibitors may improve these impairments. In this multicenter, randomized trial of patients with HFpEF (NCT03030235),
         we evaluated whether the SGLT2 inhibitor dapagliflozin improves the primary endpoint of Kansas City Cardiomyopathy Questionnaire
         Clinical Summary Score (KCCQ-CS), a measure of heart failure-related health status, at 12 weeks after treatment initiation. Secondary
         endpoints included the 6-minute walk test (6MWT), KCCQ Overall Summary Score (KCCQ-OS), clinically meaningful changes in KCCQ-CS and -OS,
         and changes in weight, natriuretic peptides, glycated hemoglobin and systolic blood pressure. In total, 324 patients were randomized to
         dapagliflozin or placebo. Dapagliflozin improved KCCQ-CS (effect size, 5.8 points (95% confidence interval (CI) 2.3–9.2, P = 0.001),
         meeting the predefined primary endpoint, due to improvements in both KCCQ total symptom score (KCCQ-TS) (5.8 points (95% CI 2.0–9.6, P = 0.003))
         and physical limitations scores (5.3 points (95% CI 0.7–10.0, P = 0.026)). Dapagliflozin also improved 6MWT (mean effect size of 20.1 m (95% CI 5.6–34.7, P = 0.007)),
         KCCQ-OS (4.5 points (95% CI 1.1–7.8, P = 0.009)), proportion of participants with 5-point or greater improvements in KCCQ-OS (odds ratio (OR)
         = 1.73 (95% CI 1.05–2.85, P = 0.03)) and reduced weight (mean effect size, 0.72 kg (95% CI 0.01–1.42, P = 0.046)). There were no significant
         differences in other secondary endpoints. Adverse events were similar between dapagliflozin and placebo (44 (27.2%) versus 38 (23.5%) patients,
         respectively). These results indicate that 12 weeks of dapagliflozin treatment significantly improved 
         patient-reported symptoms, physical limitations and exercise function and was well tolerated in chronic HFpEF.
        """ 
      ]
kw_model = KeyBERT()
keywords = []
manyKeywords = []

for i in doc:
  keywords.append(kw_model.extract_keywords(i))
  #print(i)
  manyKeywords.append(kw_model.extract_keywords(i, keyphrase_ngram_range=(1, 1), stop_words=None))

In [11]:
print(manyKeywords)

[[('cardiomyocytes', 0.4563), ('cardiomyocyte', 0.3897), ('proarrhythmic', 0.3615), ('kinase', 0.3502), ('antiarrhythmic', 0.3385)], [('cardiomyopathy', 0.4477), ('cardiometabolic', 0.3476), ('inhibitors', 0.2989), ('inhibitor', 0.2963), ('dapagliflozin', 0.2849)]]


In [12]:
keyWordArrayStorage = []

for x in manyKeywords:
    for y in x:
      output , certain = y
      keyWordArrayStorage.append(output)

print(keyWordArrayStorage)

['cardiomyocytes', 'cardiomyocyte', 'proarrhythmic', 'kinase', 'antiarrhythmic', 'cardiomyopathy', 'cardiometabolic', 'inhibitors', 'inhibitor', 'dapagliflozin']


In [ ]:
g = Graph()

In [13]:
for new in keyWordArrayStorage:
    API_ENDPOINT = "https://www.wikidata.org/w/api.php"
    query = new
    params = {
        'action': 'wbsearchentities',
        'format': 'json',
        'language': 'en',
        'search': query
    }
    r = requests.get(API_ENDPOINT, params = params)
    print(new + ': ' )
    print( r.json()['search'][0]['description'])
    print( r.json()['search'][0]['concepturi']) 
    print(' ')

cardiomyocytes: 
scientific article
http://www.wikidata.org/entity/Q61827238
 
cardiomyocyte: 
muscle cells (myocytes) that make up the cardiac muscle
http://www.wikidata.org/entity/Q1047227
 
proarrhythmic: 
scientific article
http://www.wikidata.org/entity/Q51228467
 
kinase: 
class of enzymes that catalyze the transfer of phosphate groups from high-energy, phosphate-donating molecules to specific substrates
http://www.wikidata.org/entity/Q421851
 
antiarrhythmic: 
pharmaceutical used to suppress abnormal rhythms of the heart
http://www.wikidata.org/entity/Q241549
 
cardiomyopathy: 
heart disease and a myopathy that is characterised by deterioration of the function of the heart muscle
http://www.wikidata.org/entity/Q847583
 
cardiometabolic: 
disease diagnosed by a cluster of at least 3 out of the following conditions: abdominal obesity, high blood pressure, high blood sugar, high serum triglycerides, low serum high-density lipoprotein
http://www.wikidata.org/entity/Q657193
 
inhibit

In [35]:
sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setQuery("""
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    SELECT ?label
    WHERE { <http://dbpedia.org/resource/Asturias> rdfs:label ?label }
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
    print(result["label"]["value"])

print('---------------------------')

for result in results["results"]["bindings"]:
    print('%s: %s' % (result["label"]["xml:lang"], result["label"]["value"]))

Asturias
أشتورية
Astúries
Asturie
Αστούριες
Asturien
Asturio
Asturiasko Printzerria
Asturias
Asturies
Asturias
Asturias
Asturie
アストゥリアス州
아스투리아스주
Asturië (regio)
Asturia
Астурия
Astúrias
Asturien
Астурія
阿斯图里亚斯
---------------------------
en: Asturias
ar: أشتورية
ca: Astúries
cs: Asturie
el: Αστούριες
de: Asturien
eo: Asturio
eu: Asturiasko Printzerria
es: Asturias
fr: Asturies
ga: Asturias
in: Asturias
it: Asturie
ja: アストゥリアス州
ko: 아스투리아스주
nl: Asturië (regio)
pl: Asturia
ru: Астурия
pt: Astúrias
sv: Asturien
uk: Астурія
zh: 阿斯图里亚斯
